Cell 1 – Imports + Load Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# Load data from baseline module
train = pd.read_csv("../baseline_prophet_forecast/data/train.csv")
features = pd.read_csv("../baseline_prophet_forecast/data/features.csv")
stores = pd.read_csv("../baseline_prophet_forecast/data/stores.csv")

# Merge to get a rich frame (you may not need all columns, but it's consistent)
df = (
    train
    .merge(features, on=["Store", "Date", "IsHoliday"])
    .merge(stores, on="Store")
)

df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values(["Store", "Dept", "Date"])
df.head()


Cell 2 – Build Time Series for a Store–Dept

In [ ]:
store = 1
dept = 1

ts = (
    df[(df["Store"] == store) & (df["Dept"] == dept)]
    [["Date", "Weekly_Sales"]]
    .rename(columns={"Date": "ds", "Weekly_Sales": "y"})
    .sort_values("ds")
    .set_index("ds")
)

# Ensure weekly frequency
ts = ts.asfreq("W")
ts.head()


Cell 3 – Approximate Forecast + Errors
We need a simple forecast baseline to approximate forecast errors.
Use a 4-week rolling mean:

In [ ]:
ts["forecast"] = ts["y"].rolling(4).mean()
ts["error"] = ts["y"] - ts["forecast"]

ts = ts.dropna()
ts.head()


Forecast errors will proxy the uncertainty we’re designing safety stock against.

Cell 4 – Lead-Time Demand and Safety Stock

We assume:

Lead time = 2 weeks (you can parametrize)

Target service level = 95%

Safety stock formula:

SS=z⋅σLT
	​


where:

z = service level factor (95% → 1.65)

𝜎𝐿𝑇= std dev of demand during lead time or of forecast error during LT

Here, we’ll use demand during lead time as a proxy.

In [ ]:
lead_time_weeks = 2
service_level = 0.95
z = norm.ppf(service_level)   # ≈1.645

# Demand during lead time (rolling sum over LT)
ts["lt_demand"] = ts["y"].rolling(lead_time_weeks).sum()
lt_demand = ts["lt_demand"].dropna()

mu_lt = lt_demand.mean()
sigma_lt = lt_demand.std()

safety_stock = z * sigma_lt

print(f"Mean lead-time demand: {mu_lt:.2f}")
print(f"Std dev lead-time demand: {sigma_lt:.2f}")
print(f"Safety stock for {service_level:.0%} service: {safety_stock:.2f} units")


Cell 5 – Stock-out Probability vs Inventory Level

We model lead-time demand as Normal(𝜇𝐿𝑇,𝜎𝐿𝑇) and compute:

𝑃(stockout)=𝑃(𝐷𝐿𝑇>𝐼)P(stockout)=P(DLT>I)

for different inventory levels 𝐼

In [ ]:
# Define a range of inventory levels around the mean LT demand
inventory_levels = np.linspace(mu_lt * 0.5, mu_lt * 2.0, 50)

# Stock-out probability: 1 - CDF at inventory level
stockout_probs = 1 - norm.cdf(inventory_levels, loc=mu_lt, scale=sigma_lt)

# Put into a DataFrame
risk_table = pd.DataFrame({
    "inventory_level": inventory_levels,
    "stockout_probability": stockout_probs
})

risk_table.head()


Save the table:

In [ ]:
risk_table.to_csv("inventory_risk_table_store1_dept1.csv", index=False)

Cell 6 – Plot Stock-out Curve

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(inventory_levels, stockout_probs, linewidth=2)
plt.axvline(mu_lt, color="grey", linestyle="--", label="Mean LT Demand")
plt.axvline(mu_lt + safety_stock, color="green", linestyle="--", label="Mean + Safety Stock")

plt.title(f"Stock-out Probability vs Inventory Level\nStore {store}, Dept {dept}")
plt.xlabel("Inventory Level (units over lead time)")
plt.ylabel("Stock-out Probability")
plt.ylim(0, 1)
plt.legend()
plt.tight_layout()
plt.savefig("images/stockout_curve_store1_dept1.png")
plt.show()


Cell 7 – Markdown: Interpret Results

In [ ]:
Interpretation
• Lead-time demand is summarized by a mean (μ) and uncertainty (σ).
• Safety stock is calculated as 𝑆𝑆=𝑧⋅𝜎𝐿𝑇
, where z corresponds to the target service level (e.g., 95%).
• The stock-out probability curve shows how risk decreases as inventory increases:
– Around μ, stock-out probability is ~50%.
– At μ + safety stock (for 95% SL), stock-out probability is ≈ 5%.
• In practice, this logic is applied across SKUs, sometimes with different lead times and service levels per segment (e.g., critical vs non-critical items).